In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('data.xlsx')

In [3]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [7]:
transactions = data.groupby("InvoiceNo")["Description"].apply(list)
# Преобразуем данные в множества
transactions = list(map(set, transactions))
transactions

[{'CREAM CUPID HEARTS COAT HANGER',
  'GLASS STAR FROSTED T-LIGHT HOLDER',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.',
  'SET 7 BABUSHKA NESTING BOXES',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  'WHITE METAL LANTERN'},
 {'HAND WARMER RED POLKA DOT', 'HAND WARMER UNION JACK'},
 {'ASSORTED COLOUR BIRD ORNAMENT',
  'BOX OF 6 ASSORTED COLOUR TEASPOONS',
  'BOX OF VINTAGE ALPHABET BLOCKS',
  'BOX OF VINTAGE JIGSAW BLOCKS ',
  'DOORMAT NEW ENGLAND',
  'FELTCRAFT PRINCESS CHARLOTTE DOLL',
  'HOME BUILDING BLOCK WORD',
  'IVORY KNITTED MUG COSY ',
  'LOVE BUILDING BLOCK WORD',
  "POPPY'S PLAYHOUSE BEDROOM ",
  "POPPY'S PLAYHOUSE KITCHEN",
  'RECIPE BOX WITH METAL HEART'},
 {'BLUE COAT RACK PARIS FASHION',
  'JAM MAKING SET WITH JARS',
  'RED COAT RACK PARIS FASHION',
  'YELLOW COAT RACK PARIS FASHION'},
 {'BATH BUILDING BLOCK WORD'},
 {' SET 2 TEA TOWELS I LOVE LONDON ',
  'ALARM CLOCK BAKELIKE GREEN',
  'ALARM CLOCK BAKELIKE PINK',
  'ALARM CLOCK BAKELIKE RED 

## Apriori

In [9]:
from collections import defaultdict
from itertools import combinations, chain

def generate_candidate_itemsets(itemsets):
    return set([a.union(b) for a in itemsets for b in itemsets if len(a.union(b)) == len(a) + 1])

def filter_itemsets(itemsets, transactions, min_support):
    itemset_counts = defaultdict(int)
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                itemset_counts[itemset] += 1

    num_transactions = len(transactions)
    return {itemset: support / num_transactions for itemset, support in itemset_counts.items() if support / num_transactions >= min_support}

def apriori(transactions, min_support):
    candidate_itemsets = {frozenset([item]) for transaction in transactions for item in transaction}
    frequent_itemsets = {}
    k = 1

    while candidate_itemsets:
        filtered_itemsets = filter_itemsets(candidate_itemsets, transactions, min_support)
        frequent_itemsets.update(filtered_itemsets)
        k += 1
        candidate_itemsets = generate_candidate_itemsets(filtered_itemsets)

    return frequent_itemsets


## FP-Growth (my realisation (vladimir implemented))

In [10]:
class TreeNode:
    def __init__(self, item, roots):
        self.item = item
        self.roots = roots
        self.branches = {}
        self.count = 1

    def increment(self):
        self.count += 1

def create_tree(transactions, min_support):
    header_table = {}
    for transaction in transactions:
        for item in transaction:
            header_table[item] = header_table.get(item, 0) + 1

    header_table = {item: count for item, count in header_table.items() if count >= min_support}
    if not header_table:
        return None, None

    header_table = {item: [count, None] for item, count in header_table.items()}
    tree_root = TreeNode(None, None)

    for transaction in transactions:
        transaction = {item for item in transaction if item in header_table}
        transaction = sorted(transaction, key=lambda item: header_table[item][0], reverse=True)
        current_node = tree_root
        for item in transaction:
            if item in current_node.children:
                current_node.children[item].increment()
            else:
                new_node = TreeNode(item, current_node)
                current_node.children[item] = new_node
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    header_table[item].append(new_node)
            current_node = current_node.children[item]

    return tree_root, header_table

def mine_tree(header_table, min_support, preconditions, frequent_itemsets):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])

    for item, nodes in sorted_items:
        support = sum(node.count for node in nodes[1:])
        frequent_itemsets[frozenset(preconditions + [item])] = support

        conditional_transactions = []
        for node in nodes[1:]:
            count = node.count
            transaction = []
            roots = node.roots

            while root.item is not None:
                transaction.append(root.item)
                parent = parent.parent

            for _ in range(count):
                conditional_transactions.append(transaction)

        conditional_tree, conditional_header_table = create_tree(conditional_transactions, min_support)
        if conditional_header_table is not None:
            mine_tree(conditional_header_table, min_support, preconditions + [item], frequent_itemsets)

def fpgrowth(transactions, min_support):
    min_support = min_support * len(transactions)
    tree_root, header_table = create_tree(transactions, min_support)
    if header_table is None:
        return {}

    frequent_itemsets = {}
    mine_tree(header_table, min_support, [], frequent_itemsets)
    return frequent_itemsets


## Оценим время обучения

In [21]:
import time

min_support = 0.01

start_time = time.time()

frequent_itemsets_apriori = apriori(transactions, min_support)

end_time = time.time()
apriori_time = end_time - start_time
print(f"Apriori time: {apriori_time:.2f} seconds")

Apriori time: 347.56 seconds


In [30]:
start_time = time.time()
frequent_itemsets_fpgrowth = fpgrowth(transactions, min_support)
end_time = time.time()
fpgrowth_time = end_time - start_time
print(f"FP-Growth time: {fpgrowth_time:.2f} seconds")

FP-Growth time: 2.43 seconds


### Осациацивные правила
Реализовать функцию проверки путем вывода получившихся правил как рекомендаций к текущему содержимому потребительской корзины

In [13]:
def generate_rules(frequent_itemsets, min_confidence):
    def calc_confidence(base_itemset, full_itemset):
        base_support = frequent_itemsets[base_itemset]
        full_support = frequent_itemsets[full_itemset]
        return full_support / base_support

    rules = []
    for itemset in frequent_itemsets:
        for i in range(1, len(itemset)):
            for base_itemset in map(frozenset, combinations(itemset, i)):
                confidence = calc_confidence(base_itemset, itemset)
                if confidence >= min_confidence:
                    consequence = itemset - base_itemset
                    rules.append((base_itemset, consequence, confidence))

    return rules


In [29]:
min_confidence = 0.2

rules = generate_rules(frequent_itemsets_apriori, min_confidence)

current_basket = {"LUNCH BAG RED RETROSPOT"}

print("Рекомендации:")
for base, consequence, confidence in rules:
    if base.issubset(current_basket):
        print(f"From {set(base)} to {set(consequence)} with confidence {confidence:.2f}")


Рекомендации:
From {'LUNCH BAG RED RETROSPOT'} to {'PACK OF 72 RETROSPOT CAKE CASES'} with confidence 0.22
From {'LUNCH BAG RED RETROSPOT'} to {'JUMBO STORAGE BAG SUKI'} with confidence 0.21
From {'LUNCH BAG RED RETROSPOT'} to {'JUMBO BAG PINK POLKADOT'} with confidence 0.20
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG WOODLAND'} with confidence 0.33
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG SPACEBOY DESIGN '} with confidence 0.35
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG PINK POLKADOT'} with confidence 0.38
From {'LUNCH BAG RED RETROSPOT'} to {'WHITE HANGING HEART T-LIGHT HOLDER'} with confidence 0.21
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG DOLLY GIRL DESIGN'} with confidence 0.22
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG CARS BLUE'} with confidence 0.35
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG  BLACK SKULL.'} with confidence 0.40
From {'LUNCH BAG RED RETROSPOT'} to {'CHARLOTTE BAG SUKI DESIGN'} with confidence 0.22
From {'LUNCH BAG RED RETROSPOT'} 

In [31]:
min_confidence = 0.2

rules = generate_rules(frequent_itemsets_fpgrowth, min_confidence)

current_basket = {"LUNCH BAG RED RETROSPOT"}

print("Рекомендации:")
for base, consequence, confidence in rules:
    if base.issubset(current_basket):
        print(f"From {set(base)} to {set(consequence)} with confidence {confidence:.2f}")


Рекомендации:
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG DOLLY GIRL DESIGN'} with confidence 0.22
From {'LUNCH BAG RED RETROSPOT'} to {'CHARLOTTE BAG SUKI DESIGN'} with confidence 0.22
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG WOODLAND'} with confidence 0.33
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG APPLE DESIGN'} with confidence 0.30
From {'LUNCH BAG RED RETROSPOT'} to {'RED RETROSPOT CHARLOTTE BAG'} with confidence 0.25
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG SUKI DESIGN ', 'LUNCH BAG  BLACK SKULL.'} with confidence 0.20
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG SUKI DESIGN '} with confidence 0.35
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG PINK POLKADOT', 'LUNCH BAG CARS BLUE'} with confidence 0.20
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG PINK POLKADOT', 'LUNCH BAG  BLACK SKULL.'} with confidence 0.23
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG PINK POLKADOT'} with confidence 0.38
From {'LUNCH BAG RED RETROSPOT'} to {'LUNCH BAG SPACEBOY 